In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import mean_squared_error
sns.set()

In [ ]:
# read files
input_dir='../input/tabular-playground-series-aug-2021'
train_df=pd.read_csv(os.path.join(input_dir,'train.csv'))
test_df=pd.read_csv(os.path.join(input_dir,'test.csv'))
submission_file=pd.read_csv(os.path.join(input_dir,'sample_submission.csv'))

In [ ]:
# quick look at data 
train_df.describe().T.style

In [ ]:
train_df.drop(['id'],axis=1,inplace=True)

In [ ]:
sns.histplot(data=train_df['loss'])

In [ ]:
# losses range from 0 to 42 with no negative values, it would be betterto use relu in the output layer to get positive regression values 
plt.figure(figsize=(15,5))
m=train_df['loss'].value_counts().sort_values()
sns.barplot(x=m.index,y=m.values)

In [ ]:
# Taking a small fraction of data for final evaluation 
X_train, X_test, y_train, y_test = train_test_split(train_df.iloc[:,:-1].values,train_df.iloc[:,-1].values, test_size=0.04, random_state=42)

In [ ]:
test_df.drop(['id'],axis=1,inplace=True)

In [ ]:
test_df=test_df.values

In [ ]:
def model_train(mo,epoch=1):
    fold=KFold(5,shuffle=True)
    pred=np.zeros((240000,1))
    hist=[]
    f=0
    loss=[]
    for train_id,test_id in fold.split(X_train,y_train):
        train_x=X_train[train_id]
        train_y=y_train[train_id]
        test_x=X_train[test_id]
        test_y=y_train[test_id]
        print('---------------------------validation fold : ',f,'-----------------------------------')
        mo.compile(optimizer='adam',loss=tf.keras.losses.mean_squared_error,metrics=[tf.keras.metrics.RootMeanSquaredError()])
        history=mo.fit(train_x,train_y,validation_data=(test_x,test_y),epochs=epoch)
        hist.append(history.history)
        loss.append(mo.predict(test_df))
        pred[test_id]=mo.predict(test_x)
        f+=1
    eval_(mo)
    return loss,hist,pred
def eval_(model):
    mse=mean_squared_error(y_test,model.predict(X_test).astype(int),squared=False)
    print(mse)
def to_csv(loss,filename):
    losses=np.array(loss).mean(axis=0).astype(int)
    submission_file['loss']=losses
    submission_file.to_csv(filename,index=False)

In [ ]:

model=tf.keras.Sequential([
    tf.keras.layers.Input(shape=(100,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(50,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1,activation='relu'),
    
])

In [ ]:
l0,h0,p0=model_train(model,epoch=5)
# model starts overfitting with epoch greater than 1
# RMSE score on final test is higher compared to trainand validation

In [ ]:
# Reducing number of epochs to 1
l1,h1,p1=model_train(model)

In [ ]:
to_csv(l1,'sub1.csv')

In [ ]:
model2=tf.keras.Sequential([
    tf.keras.layers.Input(shape=(100,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.BatchNormalization(),
  
    tf.keras.layers.Dense(50,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1,activation='relu'),
    
])

In [ ]:
# reducing number of layers to prevent overfitting
l2,h2,p2=model_train(model2)

In [ ]:
to_csv(l2,'sub2.csv')

In [ ]:
model3=tf.keras.Sequential([
    tf.keras.layers.Input(shape=(100,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
  
    tf.keras.layers.Dense(50,activation='relu'),
     tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1,activation='relu'),
    
])

In [ ]:
# using dropout regulrization to overcome overfitting.
l3,h3,p3=model_train(model3)

In [ ]:
to_csv(l3,'sub3.csv')

In [ ]:
model4=tf.keras.Sequential([
    tf.keras.layers.Input(shape=(100,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(50,activation='relu'),
    tf.keras.layers.BatchNormalization(),
  
    tf.keras.layers.Dense(50,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1,activation='relu'),
    
])

In [ ]:
# using a shallow neural network
l4,h4,p4=model_train(model4)

In [ ]:
to_csv(l4,'sub4.csv')

In [ ]:
def model_train_classic(model2):
    fold=KFold(5,shuffle=True)
    pred=np.zeros((240000,))

    f=0
    loss=[]
    for train_id,test_id in fold.split(X_train,y_train):
        train_x=X_train[train_id]
        train_y=y_train[train_id]
        test_x=X_train[test_id]
        test_y=y_train[test_id]
        print('---------------------------validation fold : ',f,'-----------------------------------')
       
        model2.fit(train_x,train_y)

        loss.append(model2.predict(test_df))
        pred[test_id]=model2.predict(test_x)
        f+=1
    eval_(model2)
    return loss,pred

In [ ]:
from xgboost import XGBRegressor

In [ ]:
model5=XGBRegressor(tree_method='gpu_hist')

In [ ]:
l5,p5=model_train_classic(model5)

In [ ]:
to_csv(l5,'sub5.csv')

In [ ]:
# Blending these models give a leaderboard score of 7.94122
su1=pd.read_csv('./sub1.csv')
su2=pd.read_csv('./sub2.csv')
su3=pd.read_csv('./sub3.csv.csv')
su4=pd.read_csv('./sub4.csv')
su5=pd.read_csv('./sub5.csv')
su1['loss']=0.45*su4['loss']+0.35*su3['loss']+0.1*su5['loss']+0.05*su1['loss']+0.05*su2['loss']
su1.loss=su1.loss.map(lambda x : round(x))
su1.to_csv('sub6.csv',index=False)